### Формулирование задачи
Отток клиентов является одной из главных проблем любой компании. Под оттоком клиентов понимается прекращение оплаты услуг или прекращение использования продукта компании. Процент клиентов, которые прекращают использовать продукт в течение определенного времени называется коэффициентом оттока клиентов. И если большая часть клиентов перестанет использовать продукт, то это сильно скажется на материальных потерях компании.

Основной идеей задачи является построение модели оттока клиентов, использую предоставленные данные.

#### Входные данные
* RowNumber - номер строки;
* CustomerId - уникальный идентификатор клиента;
* Surname - клиент;
* Credit Score - кредитный рейтинг;
* Geography - страна, где находится клиент;
* Gender - половой признак;
* Age - возраст клиента;
* Tenure - стаж на последнем месте работы;
* Balance - остатки на всех счетах клиента, имеющихся в банке;
* NumOfProducts - количество продуктов у клиента;
* HasCrCard - признак наличия у клиента кредитной карты;
* IsActiveMember - признак активности пользователя;
* EstimatedSalary - заработная плата клиента;
* Exited - признак того, покинул клиент банк или нет

In [1]:
import pandas as pd

In [11]:
data = pd.read_csv(r'data\Churn Modeling.csv').drop(columns='RowNumber')

Проведем первичный анализ данных. Т.е. пробежимся глазами по данным и попробуем выявить в них какие-то особенности.

Сразу наблюдается, что у нас есть столбец `RowNumber`, который не несет никакой смысловой нагрузки. Поэтому удалим его.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


In [13]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48
Exited,10000.0,2.037000e-01,0.402769,0.00,0.00,0.000000e+00,0.000000e+00,1.00
